In [1]:
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt, matplotlib.image as mpimg
%matplotlib inline
from skimage import data
from skimage.transform import rotate
import tensorflow as tf

Using TensorFlow backend.


In [4]:
labeled_images = pd.read_csv('train.csv')
images = labeled_images.iloc[:,1:]
labels = labeled_images.iloc[:,:1]
labeled_images_test = pd.read_csv('test.csv')
images_test = labeled_images_test.iloc[:,:]

m_cut_test=30000
X = images.values[:m_cut_test, :]/256.
Y = np.zeros((m_cut_test, 10), dtype = np.int32)
X_test=images.values[m_cut_test:, :]/256.
Y_test = np.zeros((images.values.shape[0]-m_cut_test, 10), dtype = np.int32)

for i in range(images.values.shape[0]):
    if i<m_cut_test:
        Y[i, labels.values[i, :]] = 1
    else:
        Y_test[i-m_cut_test, labels.values[i, :]] = 1
    
picture=np.reshape(X[:,:],(X.shape[0],28,28))
pictures2=np.zeros((X.shape[0],28,28))
for i in range(X.shape[0]):
    pictures2[i,:,:]= rotate(picture[i,:,:], angle=np.random.uniform(-20, 20),resize=False, mode='constant')

#------------for creating fully connected NN classifier---------------------
#X_augmented=np.reshape(pictures2,(X.shape[0],784))
#X_all=np.concatenate((X,X_augmented),axis=0)
X_test_all=np.reshape(X_test,(X_test.shape[0],28,28,1))
X_all=np.concatenate((picture,pictures2),axis=0)
X_all=np.reshape(X_all,(X_all.shape[0],X_all.shape[1],X_all.shape[2],1))
Y_all=np.concatenate((Y,Y))


MemoryError: 

In [2]:
#LeNet-5 with keras

def model(input_shape):
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes to make it LeNet-5 (32,32,3)
    X = ZeroPadding2D((2, 2))(X_input)

    # one-conv
    X = Conv2D(8, (5, 5), strides = (1, 1), name = 'conv0')(X)
    X = MaxPooling2D((2, 2), name='max_pool0')(X)    
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)
    
    #two-conv
    X = Conv2D(16, (5, 5), strides = (1, 1), name = 'conv1')(X)
    X = MaxPooling2D((2, 2),strides=2 ,name='max_pool1')(X)    
    X = BatchNormalization(axis = 3, name = 'bn1')(X)
    X = Activation('relu')(X)
    
    

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(120, activation='relu', name='fc0')(X)
    X = Dense(84, activation='relu', name='fc1')(X)
    X = Dense(10, activation='softmax', name='fc2')(X)
    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='HappyModel')

    return model

In [3]:
happyModel=model(input_shape=(28, 28,1))
happyModel.compile(optimizer = "Adam", loss = "logcosh", metrics = ["accuracy"])
happyModel.fit(x = X_all, y = Y_all, epochs = 10, batch_size = 16)

NameError: name 'X_all' is not defined

In [34]:
preds = happyModel.evaluate(x = X_test_all, y = Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

12000/12000 [==============================] - 2s 183us/step
Loss = 0.001076029833518143
Test Accuracy = 0.9858333333333333


In [31]:
happyModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 32, 32, 1)         0         
_________________________________________________________________
conv0 (Conv2D)               (None, 28, 28, 8)         208       
_________________________________________________________________
max_pool0 (MaxPooling2D)     (None, 14, 14, 8)         0         
_________________________________________________________________
bn0 (BatchNormalization)     (None, 14, 14, 8)         32        
_________________________________________________________________
activation_5 (Activation)    (None, 14, 14, 8)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 10, 10, 8)         1608      
__________

In [35]:
X_test_eval= images_test.values[:, :] / 256.
X_test_eval=np.reshape(X_test_eval,(X_test_eval.shape[0],28,28,1))
print(X_test_eval.shape)

(28000, 28, 28, 1)


In [36]:
Y=happyModel.predict(X_test_eval)

In [40]:
Y_predict=np.argmax(Y, axis = 1).reshape(Y.shape[0],1) 
print(Y_predict.shape)

df = pd.DataFrame(Y_predict[:,0],columns = ['Label'])
df.index = df.index + 1
df.to_csv('predict_LeNet_5.csv',index=True,index_label='ImageId')

(28000, 1)
